# Assignment: Data Wrangling
### `! git clone https://github.com/ds4e/wrangling`
### Do Q1 and Q2, and either Q3 or Q4, for a total of 3 questions.

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
   * I just basically rid, based on the Price column, of outliers beyond the .25 and .75 quantiles. I didn't really have missing values, but there were 1518 values that were beyond the lower and upper range I set. These 1518 rows were removed.
  
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? 
   * According to the code below, out of the 12925 rows, 9848 rows were missing data, or had nan, in the subject_injury column. This is a huge concern because that is 3/4 of the data with no data in that column. If we just removed these 9848 rows, we would miss all the large amount of data possible in the other columns. However, if we were thinking about this semantically, it'd be nice to know if the subject was injured, but it might not be necessary to know if the police used force overall. It depends on what column you look at that you call this large amount of missing data as "concerning". 
   * I just changed the Nans to "Unknown" without removing the 9848 rows.
   * Based on the cross tab lambda function to show how many missing values there are per feature, it appears that there's a weak pattern. `Less lethal` and `Maximal Restraint Technique` both show 100% missing values while `Firearm` and `Less Lethal Projectile` both show 0% missing data. Maybe when projectiles are involved, there's a stricter standard to report these incidents while whatever the 100% missing value rate features might have no restrictions on reporting. Somehow, it appears that the `taser` reports high amounts of missing data on injuries. It might be that subjects of tasers aren't easily injured by the taser in this dataset, but I feel like getting tased might cause a few bruises or painful days. It's interesting to me that this could happen, and the same goes for `bodily force`.

3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
   * It appears there was a weird 9 value other than 0 and 1, and there was 31 9's out of the total thousands of values. I just changed all 9's to np.nan in the column since it appeared they were either outliers or disguised unknowns.

4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)
   * I explain most of my choices in the cell 1.4 listed below, but I just replaced nans or unkonwn values like ' ' to np.nan, "eliminate" outliers by taking the lower and upper bounds then keeping inclusively what's between both. I also convert the data types to floats by coercion, though there's probably a better way to format the numbers back from original strings.

In [5]:
import numpy as np # Import the numpy package into your workspace
import matplotlib.pyplot as plt # Import matplotlib into your workspace
import pandas as pd  # Import the pandas package into your workspace

In [17]:
#Q1.1: 
airbnb_df = pd.read_csv('./data/airbnb_hw.csv', low_memory=False)

#To look at the data's properties: row x col, and column names/types: looks like Price is object type :(
print(airbnb_df.shape, '\n')
print(airbnb_df.columns, '\n')
print(airbnb_df.dtypes, '\n')

# replace $ and , - special characters removed, then coerce to numeric
airbnb_df['Price'] = airbnb_df['Price'].str.replace('$', '', regex=False)
airbnb_df['Price'] = airbnb_df['Price'].str.replace(',', '', regex=False)
airbnb_df['Price'] = pd.to_numeric(airbnb_df['Price'], errors='coerce')

#More checks on data, if any values are na (looks like no) and the .25, .75 quantiles of data
print(airbnb_df.dtypes, '\n')
missing_count = airbnb_df['Price'].isna().sum()
print(missing_count)
price_summary = airbnb_df['Price'].describe(percentiles=[0.25, 0.75])
print(price_summary)

#Use quantiles and to calculate lower and upper bounds of "legit" vs outlier data
Q1 = airbnb_df['Price'].quantile(0.25)
Q3 = airbnb_df['Price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

#Pre-check on how many outliers there are
outliers = airbnb_df[(airbnb_df['Price'] < lower_bound) | (airbnb_df['Price'] > upper_bound)]
print(len(outliers))

#Take out outliers & only keep those between upper/lower bounds
airbnb_df = airbnb_df[(airbnb_df['Price'] >= lower_bound) & (airbnb_df['Price'] <= upper_bound)]

#Post-check on how many outliers are left (should be 0)
outliers_cleaned = airbnb_df[(airbnb_df['Price'] < lower_bound) | (airbnb_df['Price'] > upper_bound)]
print(len(outliers_cleaned))

#I put na value imputation here after cleaning outliers - compute median of non-outlier data, then find new median to do imputation
airbnb_df['Price'] = airbnb_df['Price'].fillna(airbnb_df['Price'].median())

(30478, 13) 

Index(['Host Id', 'Host Since', 'Name', 'Neighbourhood ', 'Property Type',
       'Review Scores Rating (bin)', 'Room Type', 'Zipcode', 'Beds',
       'Number of Records', 'Number Of Reviews', 'Price',
       'Review Scores Rating'],
      dtype='object') 

Host Id                         int64
Host Since                     object
Name                           object
Neighbourhood                  object
Property Type                  object
Review Scores Rating (bin)    float64
Room Type                      object
Zipcode                       float64
Beds                          float64
Number of Records               int64
Number Of Reviews               int64
Price                          object
Review Scores Rating          float64
dtype: object 

Host Id                         int64
Host Since                     object
Name                           object
Neighbourhood                  object
Property Type                  object
Review Scores Rating (bin)  

In [16]:
#Q1.2: 
police_df = pd.read_csv('./data/mn_police_use_of_force.csv',low_memory=False)

#These prints tell me that there's 12925 rows, but 9848 rows with nan for subject_injury
#That's a lot, so we're probably not gonna want to delete them
print(police_df.shape)
missing_injury = police_df['subject_injury'].isna().sum()
print(missing_injury)

missing_by_group = police_df.groupby('force_type')['subject_injury'].apply(lambda x: x.isna().mean())
print(missing_by_group)

#Cross-tabulating:
cross_tab_si_ft = pd.crosstab(police_df['subject_injury'], police_df['force_type'])
print(cross_tab_si_ft)

police_df['subject_injury'] = police_df['subject_injury'].fillna('Unknown')

#Check that nans aren't there anymore:
missing_injury = police_df['subject_injury'].isna().sum()
print(missing_injury)

(12925, 13)
9848
force_type
Baton                          0.500000
Bodily Force                   0.747720
Chemical Irritant              0.892028
Firearm                        0.000000
Gun Point Display              0.259615
Improvised Weapon              0.500000
Less Lethal                    1.000000
Less Lethal Projectile         0.000000
Maximal Restraint Technique    1.000000
Police K9 Bite                 0.402597
Taser                          0.753634
Name: subject_injury, dtype: float64
force_type      Baton  Bodily Force  Chemical Irritant  Firearm  \
subject_injury                                                    
No                  0          1093                131        2   
Yes                 2          1286                 41        0   

force_type      Gun Point Display  Improvised Weapon  Less Lethal Projectile  \
subject_injury                                                                 
No                             33                 34              

In [ ]:
#1.3:
url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
pretrial_df = pd.read_csv(url,low_memory=False)

# This indicates that there are 0 "missing values" hmm
missing_releasedpretrial = pretrial_df['WhetherDefendantWasReleasedPretrial'].isna().sum()
print(missing_releasedpretrial)

# Im gonna check for disguised missing values
# It looks like there's a weird 9 value, and there's only 31 of them, so it's missing value representative (shown by the graph)
print(pretrial_df['WhetherDefendantWasReleasedPretrial'].unique())
print(pretrial_df['WhetherDefendantWasReleasedPretrial'].value_counts())

# Replace missing 9 values with np.nan:
pretrial_df['WhetherDefendantWasReleasedPretrial'] = pretrial_df['WhetherDefendantWasReleasedPretrial'].replace(9, np.nan)

# These check that there's now only 2 unique values, 0 and 1:
print(pretrial_df['WhetherDefendantWasReleasedPretrial'].unique())
print(pretrial_df['WhetherDefendantWasReleasedPretrial'].value_counts())


0
[9 0 1]
WhetherDefendantWasReleasedPretrial
1    19154
0     3801
9       31
Name: count, dtype: int64
[nan  0.  1.]
WhetherDefendantWasReleasedPretrial
1.00    19154
0.00     3801
Name: count, dtype: int64


In [61]:
#1.4
# Uncomment if you need to reload data for some reason:
# url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
# pretrial_df = pd.read_csv(url,low_memory=False)

# Check values of dataset:
print(pretrial_df['ImposedSentenceAllChargeInContactEvent'].info())
print(pretrial_df['ImposedSentenceAllChargeInContactEvent'].unique())

# There are strings, but more importantly, there are ' ' in this dataset. I'm gonna fix by replacing with np.nan:
pretrial_df['ImposedSentenceAllChargeInContactEvent'] = (
    pretrial_df['ImposedSentenceAllChargeInContactEvent']
    .str.strip()
    .replace('', np.nan)
)
print(pretrial_df['ImposedSentenceAllChargeInContactEvent'].unique())

# Now that the ' ' are gone, I'm going to coerce to numeric:
pretrial_df['ImposedSentenceAllChargeInContactEvent'] = pd.to_numeric(
    pretrial_df['ImposedSentenceAllChargeInContactEvent'], errors='coerce'
)
# print(pretrial_df['ImposedSentenceAllChargeInContactEvent'].unique())

# Gonna now find outliers:
print(pretrial_df['ImposedSentenceAllChargeInContactEvent'].describe())

Q1 = pretrial_df['ImposedSentenceAllChargeInContactEvent'].quantile(0.25)
Q3 = pretrial_df['ImposedSentenceAllChargeInContactEvent'].quantile(0.75)
IQR = Q3 - Q1

# Like in an earlier question, gonna find lower and upper bound to "eliminate" outliers - not winsorizing
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

pretrial_df = pretrial_df[
    (pretrial_df['ImposedSentenceAllChargeInContactEvent'] >= lower_bound) &
    (pretrial_df['ImposedSentenceAllChargeInContactEvent'] <= upper_bound)
]

# Now, we should have a set without "outliers", floats, and nans/other unknown values set to np.nan

count   13933.00
mean       19.90
std        63.57
min         0.00
25%         0.00
50%         2.00
75%        12.00
max      2208.00
Name: ImposedSentenceAllChargeInContactEvent, dtype: float64


**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
   * answered in comment under #2.8's section.
9.  What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

In [203]:
#2.1
# uncomment line if you need xlrd or error comes up (might need to do this in terminal if doesn't work after)
# ! pip install xlrd
file = "./data/GSAF5.xls"
shark_df = pd.read_excel(file, engine="xlrd")

#2.2
# Checking which columns are completely empty & removing them:
# empty_cols = shark_df.columns[shark_df.isna().all()]
# print(empty_cols)
shark_df = shark_df.dropna(axis=1, how="all") # I printed out that no columns were completely empty, but I just removed them with this

#2.3
# print(shark_df.describe)
years_missing = shark_df['Year'].isna().sum()
print(years_missing)
# apparently there's 2 missing values. Let's remove:
shark_df = shark_df.dropna(subset=['Year'])
years_missing = shark_df['Year'].isna().sum()
print(years_missing)
print(shark_df['Year'].dtypes) # proves values are floats or number
shark_df['Year'] = shark_df['Year'].astype(int)
print(shark_df['Year'].max() > 2025) # why are there years greater than 2025. :(
shark_df = shark_df[shark_df['Year'] <= 2025]
print(shark_df['Year'].max() > 2025) #yey
print(shark_df['Year'].min() < 1940) #nooo
shark_df = shark_df[shark_df['Year'] >= 1940]
print(shark_df['Year'].min() < 1940) #yey
print("Unique Years:", sorted(shark_df['Year'].unique()))

#2.4
# print(shark_df.describe)
print(shark_df.shape)
age_missing = shark_df['Age'].isna().sum()
print(age_missing)
# apparently there's 2994 missing values. Let's remove:
shark_df = shark_df.dropna(subset=['Age'])
age_missing = shark_df['Age'].isna().sum()
print(age_missing)
print(shark_df['Age'].dtypes) # proves values are Object-> need to change
shark_df['Age'] = pd.to_numeric(shark_df['Age'], errors='coerce')
print(shark_df['Age'].dtypes) #yay - float
age_missing = shark_df['Age'].isna().sum()
print(age_missing) #WHYY there's missing values again
shark_df = shark_df.dropna(subset=['Age'])
age_missing = shark_df['Age'].isna().sum()
print(age_missing) #yey
shark_df['Age'] = shark_df['Age'].astype(int)
print(shark_df['Age'].dtypes) #converted to int bc ages are int usually- after nans are gone
print(shark_df['Age'].min() < 1) #good it's false
print(shark_df['Age'].min() > 100) #okay that's false too

#2.5
shark_df['Sex'] = shark_df['Sex'].str.strip().str.upper()
number_male = (shark_df['Sex'] == 'M').sum()
amount_in_shark = shark_df['Sex'].notna().sum()
proportion_male = number_male / amount_in_shark
print("The proportion of males in this dataset is: ", proportion_male)

#2.6
print(shark_df['Type'].value_counts())
shark_df = shark_df[shark_df['Type'].isin(["Provoked", "Unprovoked", "Unknown"])]
print(shark_df['Type'].value_counts())
provok_number = (shark_df['Type'] == 'Unprovoked').sum()
# print(provok_number)
amount_in_shark_provok = shark_df['Type'].notna().sum()
print("Proportion of attacks unprovoked: ", provok_number/amount_in_shark_provok)

#2.7
# print(shark_df['Fatal Y/N'].value_counts()) # wow so many close values
shark_df['Fatal Y/N'] = shark_df['Fatal Y/N'].str.strip().str.upper()
# print(shark_df['Fatal Y/N'].value_counts())
# print(shark_df.loc[shark_df['Fatal Y/N'].isin(["NQ", "Y X 2"])])
shark_df['Fatal Y/N'] = shark_df['Fatal Y/N'].replace({'Y X 2': 'Y'})
shark_df['Fatal Y/N'] = shark_df['Fatal Y/N'].replace({'NQ': 'UNKNOWN'})
shark_df['Fatal Y/N'] = shark_df['Fatal Y/N'].replace({'F': 'UNKNOWN', 'M': 'UNKNOWN'})
print(shark_df['Fatal Y/N'].value_counts())

#2.8
# For p(unprovoked | male) & p(unprovoked | female): (unprovoked & sex) / sex
shark_df = shark_df[~shark_df['Sex'].isin(["LLI"])]
print(shark_df['Sex'].value_counts())
number_unprovoked_male = shark_df[(shark_df['Type'] == 'Unprovoked') & (shark_df['Sex'] == 'M')].shape[0] / shark_df[shark_df['Sex'] == 'M'].shape[0]
number_unprovoked_female = shark_df[(shark_df['Type'] == 'Unprovoked') & (shark_df['Sex'] == 'F')].shape[0] / shark_df[shark_df['Sex'] == 'F'].shape[0]
print(number_unprovoked_male, number_unprovoked_female, "It appears it's more common for sharks to attack males unprovoked than females")
# For p(fatal | unprovoked) & p(fatal | provoked): (fatal & provoked) / provoked
number_fatal_unprovoked = shark_df[(shark_df['Fatal Y/N'] == 'Y') & (shark_df['Type'] == 'Unprovoked')].shape[0] / shark_df[shark_df['Type'] == 'Unprovoked'].shape[0]
number_fatal_provoked = shark_df[(shark_df['Fatal Y/N'] == 'Y') & (shark_df['Type'] == 'Provoked')].shape[0] / shark_df[shark_df['Type'] == 'Provoked'].shape[0]
print(number_fatal_unprovoked, number_fatal_provoked, "It appears it's more fatal when sharks attack unprovoked than provoked")
# For p(fatal | male) & p(fatal | female): (fatal & sex) / sex
number_fatal_male = shark_df[(shark_df['Fatal Y/N'] == 'Y') & (shark_df['Sex'] == 'M')].shape[0] / shark_df[shark_df['Sex'] == 'M'].shape[0]
number_fatal_female = shark_df[(shark_df['Fatal Y/N'] == 'Y') & (shark_df['Sex'] == 'F')].shape[0] / shark_df[shark_df['Sex'] == 'F'].shape[0]
print(number_fatal_male, number_fatal_female, "It appears it's more fatal when sharks attack provoked given male rather than female")
# I am feeling great because males have a higher chance of dying when a shark is provoked but not so great because being attacked unprovoked is a lot worse than provoked
# For the higher male proportions in fatality, it could be because men may have higher chances of challenging animals to fight. "Cockiness" maybe

#2.9
# print(shark_df['Species '].value_counts())
shark_df['Species '] = shark_df['Species '].str.strip().str.lower()
white_shark_attacks = shark_df['Species '].str.contains(r'\bwhite\b|\bgreat white\b', na=False).sum()
print("According to math, there are ", white_shark_attacks, " Great white sharks that attacked in this data set.")

2
0
float64
True
False
True
False
Unique Years: [np.int64(1940), np.int64(1941), np.int64(1942), np.int64(1943), np.int64(1944), np.int64(1945), np.int64(1946), np.int64(1947), np.int64(1948), np.int64(1949), np.int64(1950), np.int64(1951), np.int64(1952), np.int64(1953), np.int64(1954), np.int64(1955), np.int64(1956), np.int64(1957), np.int64(1958), np.int64(1959), np.int64(1960), np.int64(1961), np.int64(1962), np.int64(1963), np.int64(1964), np.int64(1965), np.int64(1966), np.int64(1967), np.int64(1968), np.int64(1969), np.int64(1970), np.int64(1971), np.int64(1972), np.int64(1973), np.int64(1974), np.int64(1975), np.int64(1976), np.int64(1977), np.int64(1978), np.int64(1979), np.int64(1980), np.int64(1981), np.int64(1982), np.int64(1983), np.int64(1984), np.int64(1985), np.int64(1986), np.int64(1987), np.int64(1988), np.int64(1989), np.int64(1990), np.int64(1991), np.int64(1992), np.int64(1993), np.int64(1994), np.int64(1995), np.int64(1996), np.int64(1997), np.int64(1998), np.int6

**Q3.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q4.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?